In [1]:
import os
import configparser

import pandas as pd

import psycopg2

import time

In [2]:
# 쿼리 조회 시간을 체크
def my_timer(original_function):

    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_function(*args, **kwargs)
        t2 = time.time() - t1
        print('{} 함수가 실행된 총 시간: {} 초'.format(original_function.__name__, t2))
        return result

    return wrapper

@my_timer
# DB에서 데이터를 불러오기 위함
def query_select(query, params = []):

    root_path = os.path.dirname(os.getcwd())
    
    config  = configparser.ConfigParser()
    config.read(root_path + '/source/db_config.ini', encoding = 'utf-8')

    USERNAME = config['POSTGRES']['USERNAME']
    PASSWORD = config['POSTGRES']['PASSWORD']
    HOST = config['POSTGRES']['HOST']
    PORT = config['POSTGRES']['PORT']
    DATABASE = config['POSTGRES']['DATABASE']
    
    conn = psycopg2.connect(host=HOST,
                            user=USERNAME,
                            password=PASSWORD,
                            port=PORT,
                            database=DATABASE
                           )
    
    with conn.cursor() as cursor:
        
        if len(params) > 0:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
            
        headers = [x[0] for x in cursor.description]
        data = cursor.fetchall()
    
    conn.close()
       
    return pd.DataFrame(columns=headers, data=data)

In [3]:
qry = """
    create temporary table drug_pair (drug_concept_id1 int, drug_concept_id2 int);
    insert into drug_pair values (40213154,19078106),
    (19078106,40213154),
    (19009384,19030765),
    (40224172,40213154),
    (19127663,19009384),
    (1511248,40169216),
    (40169216,1511248),
    (1539463,19030765),
    (19126352,1539411),
    (1539411,19126352),
    (1332419,19126352),
    (40163924,19078106),
    (19030765,19009384),
    (19106768,40213154),
    (19075601,19126352);
    with drug_list as (
            select 
                distinct 
                drug_concept_id, 
                concept_name, 
                count(*) as cnt 
            from drug_exposure de
            join concept
            on drug_concept_id = concept_id
            where concept_id in (40213154,19078106,19009384,40224172,
                                 19127663,1511248,40169216,1539463,
                                 19126352,1539411,1332419,40163924,
                                 19030765,19106768,19075601)
            group by drug_concept_id,concept_name
            order by cnt desc),
        drugs as (
            select drug_concept_id, concept_name 
            from drug_list
        ), 
        prescription_count as (
            select drug_concept_id, cnt 
            from drug_list
        )
    select 
        a.drug_concept_id1,
        d.concept_name,
        b.cnt f_cnt,
        a.drug_concept_id2,
        c.cnt s_cnt
    from 
        drug_pair a
        left join prescription_count b
        on a.drug_concept_id1 = b.drug_concept_id
        left join prescription_count c
        on a.drug_concept_id2 = c.drug_concept_id
        inner join drugs d
        on a.drug_concept_id1 = d.drug_concept_id
    where 
        b.cnt < c.cnt
    order by f_cnt desc
"""

In [4]:
result = query_select(qry)
result

query_select 함수가 실행된 총 시간: 0.1780095100402832 초


,drug_concept_id1,concept_name,f_cnt,drug_concept_id2,s_cnt
0,19078106,hydrochlorothiazide 25 MG Oral Tablet,3669,40213154,8015
1,40224172,amlodipine 5 MG / hydrochlorothiazide 12.5 MG ...,2814,40213154,8015
2,19127663,atenolol 50 MG / chlorthalidone 25 MG Oral Tab...,2729,19009384,2825
3,40169216,120 ACTUAT fluticasone propionate 0.044 MG/ACT...,1716,1511248,1802
4,1539411,simvastatin 20 MG Oral Tablet,1318,19126352,1486
5,1332419,amlodipine 5 MG Oral Tablet,1247,19126352,1486
6,40163924,24 HR metformin hydrochloride 500 MG Extended ...,1235,19078106,3669
7,19030765,1 ML epoetin alfa 4000 UNT/ML Injection [Epogen],1214,19009384,2825
8,19106768,hydrochlorothiazide 12.5 MG Oral Tablet,1199,40213154,8015
9,19075601,clopidogrel 75 MG Oral Tablet,1164,19126352,1486
